In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from rdkit.Chem import PandasTools, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn import pipeline as pl
from sklearn import preprocessing as pp
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
import graphviz 


### Dataset initiation
Initiate and prepare the dataset with descriptors to make them usable for the ML-model. 

In [3]:
dataset_molecules = pd.read_csv('tested_molecules_v2.csv')
PandasTools.AddMoleculeColumnToFrame(dataset_molecules, smilesCol='SMILES')
dataset_molecules.head()


,SMILES,ALDH1_inhibition,ROMol
0,[NH3+]CCSSCC[NH3+],0,<rdkit.Chem.rdchem.Mol object at 0x0000016C2C6...
1,[NH3+]CCC[NH2+]CCCC[NH2+]CCC[NH3+],0,<rdkit.Chem.rdchem.Mol object at 0x0000016C2C6...
2,[NH3+]CCCCCCCCCC[NH3+],0,<rdkit.Chem.rdchem.Mol object at 0x0000016C2C6...
3,[NH3+]CCSSCC[NH3+],0,<rdkit.Chem.rdchem.Mol object at 0x0000016C2C6...
4,ClCC[NH+](CCCl)CCCl,0,<rdkit.Chem.rdchem.Mol object at 0x0000016C2C6...


In [4]:

descriptors_labels = [n[0] for n in Descriptors._descList[:]]
calc = MoleculeDescriptors.MolecularDescriptorCalculator(descriptors_labels)
rdkit_desc = [calc.CalcDescriptors(m) for m in dataset_molecules["ROMol"]]
dataset_descriptors = pd.DataFrame(rdkit_desc, columns=[descriptors_labels])
dataset_descriptors = dataset_descriptors[['MaxAbsEStateIndex',
 'MinAbsEStateIndex',
 'MinEStateIndex',
 'BCUT2D_CHGHI',
 'BCUT2D_CHGLO',
 'BCUT2D_LOGPLOW',
 'BCUT2D_MRLOW',
 'SMR_VSA1',
 'SMR_VSA5',
 'SlogP_VSA2',
 'EState_VSA10',
 'VSA_EState2',
 'FractionCSP3',
 'NumAliphaticRings']]
X_train, X_test, y_train, y_test = train_test_split(dataset_descriptors, dataset_molecules['ALDH1_inhibition'], test_size=0.5, random_state=42)
dataset_descriptors.head()

,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPLOW,BCUT2D_MRLOW,SMR_VSA1,SMR_VSA5,SlogP_VSA2,EState_VSA10,VSA_EState2,FractionCSP3,NumAliphaticRings
0,3.733529,1.036517,1.036517,1.865112,-1.907393,-2.514714,-0.360469,11.467335,0.000000,24.595220,0.0,0.000000,1.0,0
1,3.831341,1.078701,1.078701,1.940948,-2.003261,-2.860321,-0.666457,22.100912,25.683286,39.268538,0.0,0.000000,1.0,0
2,3.837048,1.112603,1.112603,1.908706,-1.984755,-2.495713,-0.368558,11.467335,51.366573,13.089513,0.0,0.000000,1.0,0
3,3.733529,1.036517,1.036517,1.865112,-1.907393,-2.514714,-0.360469,11.467335,0.000000,24.595220,0.0,0.000000,1.0,0
4,5.560741,0.683642,0.683642,2.035557,-2.143624,-3.126526,-0.892277,4.899910,0.000000,37.274280,0.0,1.388889,1.0,0


### Introducing a Pipeline with minmax scaler and Desciscion tree.

In [5]:
scaler = pp.MinMaxScaler()
pca = PCA()
dtree = tree.DecisionTreeClassifier(max_features=None, max_depth=None, max_leaf_nodes=None)

pipeline = pl.Pipeline(steps=[("sc", scaler), ("pca", pca), ("dtreeCLF", dtree)])
cross_score = cross_val_score(pipeline, X_train, y_train, cv=10)
print("Cross score:") 
print(cross_score)
pipeline.fit(X_train, y_train)
print("score:")
print(pipeline.score(X_test, y_test))

Cross score:
[0.92 1.   0.98 0.98 0.98 0.92 0.98 0.96 0.96 0.92]
score:
0.964


### Creating a physical tree

In [6]:
dot_data = tree.export_graphviz(dtree, out_file=None, feature_names=dataset_descriptors.columns, class_names=['1', '0'])
graph = graphviz.Source(dot_data) 
graph.render("descision_tree") 

'descision_tree.pdf'

### Analysing the model

In [7]:
y_true = y_test
y_pred = pipeline.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
specificity = tn / (tn+fp)
sensitivity = tp/(tp+fp) 
accuracy = (tp+tn)/(tp+tn+fp+fn)
print(" |+\t|-\t|\n+|{:.0f}\t|{:.0f}\t|\n-|{:.0f}\t|{:.0f}\t|".format(tp, fp, fn, tn))
print("Specificiteit: {:.2f} Sensitivity: {:.2f} Accuracy {:.2f}".format(specificity, sensitivity, accuracy))

 |+	|-	|
+|123	|8	|
-|10	|359	|
Specificiteit: 0.98 Sensitivity: 0.94 Accuracy 0.96
